In [ ]:
%load_ext autoreload
%autoreload
from factory import *
import torch
#from model import Unet
from catalyst.dl.callbacks import CriterionCallback, EarlyStoppingCallback
from catalyst.dl.runner import SupervisedRunner
from pytorch_toolbelt import losses as L
import collections
from pytorch_toolbelt.utils.catalyst import * 
from metrics import *
import custom_tta as tta
from pytorch_toolbelt.inference.tiles import *
import matplotlib.pyplot as plt
from viz_utils import *
from tqdm import tqdm
import segmentation_models_pytorch as smp
from tqdm import tqdm_notebook
import pretrainedmodels
from pytorch_toolbelt.inference import tta
from itertools import combinations
from catalyst import utils
import seaborn as sns
%matplotlib inline

In [ ]:
all_encoder_names = [
            #'resnext50_32x4d',
            #'se_resnext101_32x4d', 
            #'efficientnet-b5',
            'resnext101_32x8d',
            #'se_resnet50']
all_experiment_names = [
            #'resnext50_32x4d_simple',
            #'se_resnext101_32x4d_simple', 
            #'efficientnet-b5_simple', 
            'resnext101_32x8d_with_mask_and_boundaries',
            #'se_resnet50_simple'
            ]
all_log_dirs = [
            #'logs/resnext50_32x4d_simple',
            #'logs/se_resnext101_32x4d_simple', 
            #'logs/efficientnet-b5_simple',
            'logs/resnext101_32x8d_with_mask_and_boundaries',
            #'logs/se_resnet50_simple'
            ]
#output_channels = [4,4,4,4,4]
output_channels = [6]   
train_df_path = 'data/train.csv'
data_folder = "data/train_images/"
test_data_folder = "data/test_images/"
val_output_folder = "data/validation_predictions"
sample_submission_path = 'data/sample_submission.csv'

In [ ]:
class TestDataset(Dataset):
    '''Dataset for test prediction'''
    def __init__(self, root, fnames, transforms):
        self.root = root
        self.fnames = fnames.tolist()
        self.num_samples = len(self.fnames)
        self.transforms = transforms

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname)
        image = cv2.imread(path)
        image = self.transforms(image=image)["image"]
        image = torch.from_numpy(image.transpose((2, 0, 1))).float()
        return {'features':image}

    def __len__(self):
        return self.num_samples

In [ ]:
sample_submission = pd.read_csv(sample_submission_path)
test_images = np.unique([x.split('_')[0] for x in sample_submission.ImageId_ClassId.values])
test_loader = DataLoader(
    TestDataset(test_data_folder, test_images, validation_augmentations()),
    batch_size=32,
    num_workers=4,
    shuffle=False,   
    )

In [ ]:
device = utils.get_device()
print(f"device: {device}")

In [ ]:
runner = SupervisedRunner(device=device, input_key="features")
predictions_test = []
for idx in range(len(all_encoder_names)):
    print('Infering test using {}'.format(all_encoder_names[idx]))
    runner = SupervisedRunner(device=device, input_key='features')
    model = smp.Unet(classes=output_channels[idx], 
                    encoder_name=all_encoder_names[idx],
                    encoder_weights=None)
    predictions_test.append(runner.predict_loader(
            model=model,
            loader=test_loader,
            resume=f"{all_log_dirs[idx]}/checkpoints/best.pth",
            verbose=True,
        ))

In [ ]:
images_id = []
masks_rle = []
min_area = [600, 600, 1000, 2000]
for idx in tqdm(range(len(test_images))):
    image_pred = np.stack([nn.Sigmoid()(torch.from_numpy(data_[idx][:4,...])).numpy() for data_ in predictions_test]).mean(axis=0)
    for defect_type in range(4):
        image_pred_bin = image_pred[defect_type,...]>0.5
        if image_pred_bin.sum() < min_size[defect_type]:
            masks_rle.append('')
        else:
            masks_rle.append(mask2rle(image_pred_bin))
        images_id.append(test_images[idx]+'_{}'.format(defect_type+1))     

In [ ]:
submission = pd.DataFrame({'ImageId_ClassId':images_id,'EncodedPixels':masks_rle})
submission.head()

In [ ]:
classifications = pd.read_csv('classificaion_test/effnetb3_3folds_true_validation_tta_fliplr.csv')
classifications.columns.values[1] = 'has_mask'

In [ ]:
submission = submission.merge(classifications)

In [ ]:
submission.loc[submission.has_mask==0, 'EncodedPixels'] = ''

In [ ]:
submission.loc[:,['ImageId_ClassId', 'EncodedPixels']].to_csv('./submission_vD.tsv',index=False)